In [10]:
%load_ext autoreload
%autoreload 2

# https://scipy-cookbook.readthedocs.io/items/bundle_adjustment.html
import urllib
import bz2
import os
import numpy as np

BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/ladybug/"
FILE_NAME = "problem-49-7776-pre.txt.bz2"
URL = BASE_URL + FILE_NAME

if not os.path.isfile(FILE_NAME):
    urllib.request.urlretrieve(URL, FILE_NAME)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def read_bal_data(file_name):
    with bz2.open(file_name, "rt") as file:
        n_cameras, n_points, n_observations = map(
            int, file.readline().split())

        camera_indices = np.zeros(n_observations, dtype=int)
        points_2d = np.zeros((n_cameras, n_points, 1, 2))

        for i in range(n_observations):
            camera_index, point_index, x, y = file.readline().split()
            points_2d[int(camera_index), int(point_index)] = [float(x), float(y)]

        calib = {}
        camera_params = np.empty(n_cameras * 9)
        for i in range(n_cameras * 9):
            camera_params[i] = float(file.readline())
        camera_params = camera_params.reshape((n_cameras, -1))

        points_3d = np.empty(n_points * 3)
        for i in range(n_points * 3):
            points_3d[i] = float(file.readline())
        points_3d = points_3d.reshape((n_points, 1, -1))

    return camera_params, points_3d, points_2d

In [12]:
camera_params, _, points2d = read_bal_data(FILE_NAME)


In [13]:
'''
cam.rvec = camera_params[0:3]
cam.tvec = camera_params[3:6]
if update_intrinsic:
    cam.fx = camera_params[6]
    cam.fy = camera_params[7]
if update_distort:
    cam.distort  = camera_params[8:13]
'''
import cv2
calib = dict()
for cid, params in enumerate(camera_params):
    calib[cid] = {"R": cv2.Rodrigues(params[0:3])[0], "tvec": params[3:6], "intr": np.array([[params[6], 0, 0], [0, params[7], 0], [0, 0, 1]]), "distort":np.zeros(5)}

In [14]:
from pyba.CameraNetwork import CameraNetwork
camNet = CameraNetwork(points2d=points2d, calib=calib)

In [27]:
np.mean(camNet.reprojection_error())

1.2810801193150386e+19

In [26]:
from pyba.pyba import bundle_adjust
bundle_adjust(camNet, update_distort=False, update_intrinsic=True)

7776 1000.0
(3936, 2) (3936,)
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         2.2936e+11                                    3.84e+82    
       1              8         2.2841e+11      9.55e+08       3.86e+05       3.83e+82    
       2              9         2.2626e+11      2.14e+09       5.98e+05       3.80e+82    
       3             10         2.2248e+11      3.79e+09       1.43e+06       3.75e+82    
       4             11         2.1533e+11      7.15e+09       3.88e+06       1.93e+83    
       5             13         2.1204e+11      3.29e+09       1.93e+06       1.02e+84    
       6             15         2.1030e+11      1.74e+09       8.90e+05       2.53e+84    
       7             16         2.0716e+11      3.15e+09       1.72e+06       2.58e+85    
       8             26         2.0716e+11      0.00e+00       0.00e+00       2.58e+85    
`xtol` termination condition is satisfied.
Function evaluati

 active_mask: array([0., 0., 0., ..., 0., 0., 0.])
        cost: 207156693990.43845
         fun: array([ 8.01166720e+01, -2.58967776e+02,  1.61132770e+02, ...,
       -1.87892645e+01,  2.13980907e-01, -1.05351745e+01])
        grad: array([-7.03873018e+12, -2.35965488e+10,  3.80007399e+10, ...,
       -1.49380809e+01,  2.25805910e+00,  1.52565384e+02])
         jac: <7872x3637 sparse matrix of type '<class 'numpy.float64'>'
	with 125952 stored elements in Compressed Sparse Row format>
     message: '`xtol` termination condition is satisfied.'
        nfev: 26
        njev: 8
  optimality: 2.575039144953272e+85
      status: 3
     success: True
           x: array([ 1.57487309e-02, -9.72245515e-03, -4.61050474e-03, ...,
       -3.43200687e+01,  2.90746037e+03, -4.59820635e+01])